In [1]:
# ✅ Step 2: Clean & Normalize Luma Event Data
import pandas as pd
from datetime import datetime
import re

# Load the raw CSV
df = pd.read_csv("01_raw_luma_events.csv")

# 🗓 Normalize date to DD-MM-YYYY format
def clean_date(date_str):
    try:
        # Handle formats like "Jun 24" or "June 24"
        match = re.match(r"([A-Za-z]+)\s+(\d{1,2})", date_str)
        if match:
            month = match.group(1)
            day = match.group(2)
            year = datetime.now().year
            parsed = datetime.strptime(f"{day} {month} {year}", "%d %b %Y")
            return parsed.strftime("%d-%m-%Y")
        return ""
    except:
        return ""

df["event_date"] = df["event_date"].apply(clean_date)

# 🌍 Normalize locations
def clean_location(loc):
    if pd.isna(loc):
        return "Online"
    loc = loc.lower()
    if "online" in loc or "zoom" in loc or "link" in loc:
        return "Online"
    elif "hybrid" in loc:
        return "Hybrid"
    return "In-person"

df["location"] = df["location"].apply(clean_location)

# Save the cleaned CSV
df.to_csv("02_cleaned_luma_events.csv", index=False)
df.head()


,event_title,platform,topic_tags,event_type,event_date,location,speakers_orgs,url,audience_segment
0,The AI Shift in B2C | From User Experience to ...,Luma,NaN,Meetup,,In-person,NaN,https://lu.ma/fqrhf805,NaN
1,Agara Walks & Filter Kaapi - All Things Health...,Luma,NaN,Meetup,,In-person,NaN,https://lu.ma/dbiihmyw,NaN
2,AI Day Bengaluru,Luma,NaN,Meetup,,In-person,NaN,https://lu.ma/acvb1fkb,NaN
3,VIBE-ATHON: Where AI Meets Firebase Magic ⚡,Luma,NaN,Meetup,,In-person,NaN,https://lu.ma/y1arivc5,NaN
4,Build Your Own DailySync Agent,Luma,NaN,Meetup,,In-person,NaN,https://lu.ma/lwl5dh24,NaN


In [2]:
from bs4 import BeautifulSoup
import requests
import time

# Reload raw dataframe
df = pd.read_csv("eventradar_raw_luma.csv")

# Add new columns
df["event_date"] = ""
df["location"] = ""
df["event_type"] = ""

# Loop over each URL
for i, row in df.iterrows():
    try:
        event_url = row["url"]
        print(f"Scraping: {event_url}")
        driver.get(event_url)
        time.sleep(2)  # Short delay
        
        soup = BeautifulSoup(driver.page_source, "lxml")
        
        # 🎯 Get event date
        date_block = soup.find("div", class_="date")
        if date_block:
            df.at[i, "event_date"] = date_block.text.strip()

        # 🌍 Get location
        loc_block = soup.find("div", string=lambda text: text and ("Online" in text or "Singapore" in text))
        if loc_block:
            df.at[i, "location"] = loc_block.text.strip()
        else:
            df.at[i, "location"] = "Online"  # Fallback
        
        # 🔤 Infer event type
        title = row["event_title"].lower()
        if "workshop" in title:
            df.at[i, "event_type"] = "Workshop"
        elif "panel" in title:
            df.at[i, "event_type"] = "Panel"
        elif "talk" in title:
            df.at[i, "event_type"] = "Talk"
        else:
            df.at[i, "event_type"] = "Event"
        
    except Exception as e:
        print(f"Error scraping {row['url']}: {e}")
        continue

# Save intermediate cleaned file
df.to_csv("eventradar_cleaned_luma.csv", index=False)
df.head()


Scraping: https://lu.ma/vynb9gp5
Error scraping https://lu.ma/vynb9gp5: name 'driver' is not defined
Scraping: https://lu.ma/gk1ezao7
Error scraping https://lu.ma/gk1ezao7: name 'driver' is not defined
Scraping: https://lu.ma/djifk2z0
Error scraping https://lu.ma/djifk2z0: name 'driver' is not defined
Scraping: https://lu.ma/yre3ad0m
Error scraping https://lu.ma/yre3ad0m: name 'driver' is not defined
Scraping: https://lu.ma/4pxgchp4
Error scraping https://lu.ma/4pxgchp4: name 'driver' is not defined
Scraping: https://lu.ma/hqtfxq61
Error scraping https://lu.ma/hqtfxq61: name 'driver' is not defined
Scraping: https://lu.ma/IgnitionWeek_NetworkSchool
Error scraping https://lu.ma/IgnitionWeek_NetworkSchool: name 'driver' is not defined
Scraping: https://lu.ma/1jw5gmgg
Error scraping https://lu.ma/1jw5gmgg: name 'driver' is not defined
Scraping: https://lu.ma/297oglj1
Error scraping https://lu.ma/297oglj1: name 'driver' is not defined
Scraping: https://lu.ma/2wphbx55
Error scraping https:/

,event_title,platform,event_type,event_date,location,url,speakers_orgs,topic_tags,audience_segment
0,"AIGents of Change #4: Media, Authorship & AI",Luma,,,,https://lu.ma/vynb9gp5,NaN,NaN,NaN
1,Rise & Walk - Saturday June 21st @Henderson Wa...,Luma,,,,https://lu.ma/gk1ezao7,NaN,NaN,NaN
2,The Saturday Book Club June 2025 Session,Luma,,,,https://lu.ma/djifk2z0,NaN,NaN,NaN
3,Women in AI: Idea to working MVP in 2 hours!,Luma,,,,https://lu.ma/yre3ad0m,NaN,NaN,NaN
4,Conversations on Caregiving: Envisioning the F...,Luma,,,,https://lu.ma/4pxgchp4,NaN,NaN,NaN
